Поиск трастовых сейлеров 

В ходе проекта были выполнены следующие задачи:

Заключение контрактов с трастовыми сейлерами:
Контракты заключаются на особых условиях, предусматривающих предоставление скидок. Трастовые сейлеры представляют собой надежных партнеров, которые соответствуют установленным критериям.

Определение цели проекта:
Основной целью стало выявление новых трастовых сейлеров из новых стран для дальнейшего сотрудничества на выгодных условиях.

Формирование критериев отбора:
Для определения трастовых сейлеров были установлены следующие критерии:

1.Количество устройств (cn) должно быть более 20.
С момента регистрации должно пройти не менее трех месяцев.
2.Анализ данных на определенную дату:
Поиск трастовых сейлеров был выполнен на основе данных, актуальных на 8 октября 2024 года. Были проанализированы пользователи и их соответствие заданным критериям, а также распределение по странам.

3.Использование географического фильтра:
Для анализа рассматривались только пользователи, у которых хотя бы одно устройство (cn) зарегистрировано в одной из заданных стран, а остальные устройства могли находиться в других странах. Это условие введено для обеспечения гибкости в управлении устройствами.

4.Обеспечение мобильности устройств:
Наличие хотя бы одного устройства в целевой стране позволяет сейлеру оперативно перенести остальные устройства в нужные регионы при необходимости. Такой подход способствует более эффективному сотрудничеству с сейлерами, обеспечивая возможность быстрого реагирования на изменения рынка и бизнес-задач.

Итогом проекта стало выявление потенциальных трастовых сейлеров, которые могут стать надежными партнерами и расширить географическое присутствие компании.

In [ ]:
import os
from clickhouse_driver import Client
import pandas as pd

# Получаем пароль из переменной окружения
password = os.getenv("CLICKHOUSE_PASSWORD")

# Проверяем, что пароль не является None
if password is None:
    raise ValueError("Переменная окружения CLICKHOUSE_PASSWORD не установлена")

# Устанавливаем соединение с ClickHouse
client = Client(
    host='ch.infra.zprx.net',
    port=2044,
    user='zprx_analytics',
    password=password,
    secure=True,
    verify=False,
    database='zprx'
)

try:
    # Основной SQL-запрос
    query = """
    WITH user_registration AS (
        SELECT 
            d.user_id AS user_id, 
            d.email AS email, 
            d.telegram AS telegram, 
            b.cn AS cn,
            MIN(toDate(d.created_dt)) AS registration
        FROM 
            zprx_ds.users AS d
        JOIN 
            zprx_ds.cns AS b 
        ON 
            d.user_id = b.user_id
        WHERE 
            b.deleted_dt IS NULL -- Учитываем только активные устройства
            AND d.user_id NOT IN (
                -- Исключаем трастовых сейлеров, данные о которых уже есть у маркетологов
                'YGnEYF7BLE', 'nq0V2Rh4j9', 'fqbCjVym56', 'Utemmbm8E7',
                '9422KDjepC', 'KAJMldDCol', 'DJsd1UyirG', 'WBEkRhYfhr',
                'rfS4SKFwW7', 'O7LXGaowdM', 'TgMnS14ueZ', 'TojiO9AXzV',
                'M5meGKFUPG', 'Rik8HED9s1', 'SOc4n9u3LK', 'KrtVEb5BwY',
                'mJomBIILXc', 'jozdeZK15i', 'neHY1ORVJn', 'lPrMblRSx0',
                'EFArkRhhTT', 'sqvkguOqRL', 'PSkHiiDQvH', 'FGZiDLiK6b',
                'QaYXD7ypWp', 'eH3dBXaDlh', 'FvACQFfGpM', '1Kwh22qFKE',
                'nHgwHPLbGO'
            )
        GROUP BY 
            d.user_id, d.email, d.telegram, b.cn
    ),
    user_total_cn AS (
        SELECT 
            user_id, COUNT(DISTINCT cn) AS total_cnt_cn 
        FROM 
            zprx_ds.cns
        WHERE 
            deleted_dt IS NULL -- Учитываем только активные устройства
        GROUP BY 
            user_id
    ),
    final_cn AS (
        SELECT 
            a.user_id, a.email, a.telegram, a.cn, b.total_cnt_cn, 
            dateDiff('month', MIN(a.registration), now()) AS months_since_registration
        FROM 
            user_registration a
        JOIN 
            user_total_cn b ON a.user_id = b.user_id
        GROUP BY 
            a.user_id, a.email, a.telegram, a.cn, b.total_cnt_cn
        HAVING 
            b.total_cnt_cn >= 20 -- Фильтр: устройства более 20
            AND months_since_registration >= 3 -- Фильтр: регистрация более 3 месяцев назад
        ORDER BY 
            b.total_cnt_cn DESC
    )
    SELECT 
        a.user_id, 
        a.email, 
        a.telegram,  
        geoip2_country(b.client_ip) AS country_ip, 
        COUNT(DISTINCT b.cn) AS cnt_country
    FROM 
        final_cn a
    LEFT JOIN 
        zproxy_reqs_v1__merge b ON a.cn = b.cn
    WHERE 
        toDate(ts) = '2024-10-08' -- Анализируем данные на конкретную дату
        AND geoip2_country(b.client_ip) IN (
            -- Страны, где хотя бы одно устройство должно быть зарегистрировано
            'Croatia', 'Denmark', 'France', 'Georgia', 'Hungary', 'Italy', 
            'Luxembourg', 'Norway', 'Portugal', 'Slovakia', 'Sweden', 'Switzerland',
            'Bangladesh', 'China', 'Israel', 'Pakistan', 'Singapore', 'Australia',
            'New Zealand', 'Mexico', 'Argentina', 'Colombia', 'Paraguay', 
            'Nigeria', 'Russia'
        )
    GROUP BY 
        a.user_id, a.email, a.telegram, total_cnt_cn, country
    ORDER BY 
        cnt_cn DESC
    """

    # Выполняем запрос и сохраняем результат в DataFrame
    result = client.execute(query)
    trusted_sellers_analysis = pd.DataFrame(
        result, 
        columns=['user_id', 'email', 'telegram', 'country_ip', 'cnt_country']
    )

finally:
    # Закрываем соединение
    client.disconnect()


In [5]:
trusted_sellers_analysis.head(15)


,user_id,email,telegram,country_ip,cnt_country
0,uFyoy1fHAI,*****,*****,Colombia,117
1,NOP9RAwaNo,*****,*****,Colombia,81
2,LtQv61YpPE,*****,*****,France,38
3,wclwKqEfw3,*****,*****,Russia,35
4,jWLnXUReY3,*****,*****,Russia,24
5,mbyHGJOnfO,*****,*****,Italy,21
6,jRgifNQVyD,*****,*****,Russia,18
7,MDeBanQqJ4,*****,*****,France,10
8,AFMJgAyp63,*****,*****,Israel,9
9,MDeBanQqJ4,*****,*****,Italy,7
